Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [328]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from functools import reduce

In [329]:
# Define the problem dimensions
num_points = 100
num_sets = num_points
density = 0.3
problem_dim = num_points
fitness_counter = 0



In [330]:
def make_set_covering_problem(num_points, num_sets, density):
    seed(num_points * 2654435761 + num_sets + density)
    sets = sparse.lil_matrix((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets - 1), p] = True
    return sets

In [331]:
# Function to evaluate the fitness of a solution
def fitness2(state, problem):
    cost = state.count('1')
    global fitness_counter
    fitness_counter = fitness_counter + 1

    covered_items = reduce(
        np.logical_or,
        [problem[i, :].toarray()[0] for i, t in enumerate(state) if t == '1'],
        np.array([False for _ in range(problem_dim)]),
    )

    valid = np.all(covered_items)

    return valid, cost

In [332]:
def initialize_solution(num_sets):

    cap = 8 # limit the initial cost of generation 0

    num_selected_sets = min(cap, num_sets)  # Cap the maximum selected sets to 30
    selected_sets = [1] * num_selected_sets + [0] * (num_sets - num_selected_sets)
    shuffle(selected_sets)  # Shuffle the selected sets to randomize them
    selected_sets = ''.join(map(str, selected_sets))
    return selected_sets

In [333]:
# Main GA function
def genetic_algorithm(problem, max_generations, population_size):
    best_solution = None
    best_fitness = float('inf')
    population = []

    for _ in range(population_size):
        population.append(initialize_solution(num_sets))

    for generation in range(max_generations):
        new_population = []

        for _ in range(population_size):
            parent1_index, parent2_index = np.random.choice(range(population_size), size=2, replace=False)
            parent1 = population[parent1_index]
            parent2 = population[parent2_index]

            crossover_point = randint(1, num_sets - 1)
            child = parent1[:crossover_point] + parent2[crossover_point:]

            # Introduce mutation
            mutation_point = randint(0, num_sets - 1)
            mutation = random() < 0.1
            child = child[:mutation_point] + ('1' if mutation else '0') + child[mutation_point + 1:]

            valid, cost = fitness2(child, problem)

            new_population.append(child)

            if valid and cost < best_fitness:
                best_solution = child
                best_fitness = cost

        population = new_population

        print(f"Generation {generation}: Best solution = {best_solution}, Cost = {best_fitness}")

    return best_solution, best_fitness

In [334]:
""" # Generate the initial solution
initial_solution = initialize_solution(num_sets)

# Print the initial solution
print("Initial solution:", initial_solution)

initial_solution_coverage = [x[i, :].toarray()[0] for i, t in enumerate(initial_solution) if t == '1']

# Calculate the overall coverage
overall_coverage = sum(reduce(np.logical_or, initial_solution_coverage, np.array([False for _ in range(problem_dim)])))

# Print the overall coverage
print("Overall coverage by the initial solution:", overall_coverage)

initial_solution_valid, initial_solution_cost = fitness2(initial_solution, x)
print("Initial solution cost:", initial_solution_cost)
print("The solution is valid:", initial_solution_valid)
 """

' # Generate the initial solution\ninitial_solution = initialize_solution(num_sets)\n\n# Print the initial solution\nprint("Initial solution:", initial_solution)\n\ninitial_solution_coverage = [x[i, :].toarray()[0] for i, t in enumerate(initial_solution) if t == \'1\']\n\n# Calculate the overall coverage\noverall_coverage = sum(reduce(np.logical_or, initial_solution_coverage, np.array([False for _ in range(problem_dim)])))\n\n# Print the overall coverage\nprint("Overall coverage by the initial solution:", overall_coverage)\n\ninitial_solution_valid, initial_solution_cost = fitness2(initial_solution, x)\nprint("Initial solution cost:", initial_solution_cost)\nprint("The solution is valid:", initial_solution_valid)\n '

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [335]:
# Generate the set covering problem
x = make_set_covering_problem(num_points, num_sets, density)

# Call the genetic algorithm
max_generations = 20
population_size = 100
best_solution, best_fitness = genetic_algorithm(x, max_generations, population_size)

print("Number of points", num_points)
print("Density", density)
print("Max Generations", max_generations)
print("Population size", population_size)

# Print the best solution and its fitness
# print("Best solution:", best_solution)
print("Best fitness (cost):", best_fitness)
print("Number of fitness evaluations:", fitness_counter)

# Calculate the overall coverage achieved by the best solution
best_solution_coverage = [x[i, :].toarray()[0] for i, t in enumerate(best_solution) if t == '1']
best_solution_overall_coverage = sum(reduce(np.logical_or, best_solution_coverage, np.array([False for _ in range(problem_dim)])))
print("Overall coverage by the best solution:", (best_solution_overall_coverage/num_points)*100, "%")
print("Best solution cost:", best_fitness)

Generation 0: Best solution = 0000000000000000010001000000100000000000000010000000001000001000000000000000010000000001000000000001, Cost = 9
Generation 1: Best solution = 0000000000000000010001000000100000000000000010000000001000001000000000000000010000000001000000000001, Cost = 9
Generation 2: Best solution = 0000000000000000010001000000100000000000000010000000001000001000000000000000010000000001000000000001, Cost = 9
Generation 3: Best solution = 0000000000000000010001000000100000000000000010000000001000001000000000000000010000000001000000000001, Cost = 9
Generation 4: Best solution = 0000000000000000010001000000100000000000000010000000001000001000000000000000010000000001000000000001, Cost = 9


Generation 5: Best solution = 0000000000000000010001000000100000000000000010000000001000001000000000000000010000000001000000000001, Cost = 9
Generation 6: Best solution = 0000000000000000010001000000100000000000000010000000001000001000000000000000010000000001000000000001, Cost = 9
Generation 7: Best solution = 0000000000000000010001000000100000000000000010000000001000001000000000000000010000000001000000000001, Cost = 9
Generation 8: Best solution = 0000000000000000010001000000100000000000000010000000001000001000000000000000010000000001000000000001, Cost = 9
Generation 9: Best solution = 0000000000000000010001000000100000000000000010000000001000001000000000000000010000000001000000000001, Cost = 9
Generation 10: Best solution = 0000000000000000010001000000100000000000000010000000001000001000000000000000010000000001000000000001, Cost = 9
Generation 11: Best solution = 0000000000000000010001000000100000000000000010000000001000001000000000000000010000000001000000000001, Cost = 9
Generation 